In [ ]:
#Libs
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
acceptance = pd.read_csv(r'Globepay Acceptance Report - Globepay Acceptance Report.csv')
chargeback = pd.read_csv(r'Globepay Chargeback Report - Globepay Chargeback Report.csv')
data = pd.merge(acceptance, chargeback, how = 'left', on = 'external_ref')
data.info()

In [ ]:
#Check cols for being duplicates
data['check_status'] = data['status_x'] == data['status_y']
data['check_src'] = data['source_x'] == data['source_y']

In [ ]:
print(data['check_status'].value_counts())
print(data['check_src'].value_counts())

In [ ]:
#Drop duplicate columns
data.drop(columns = ['status_y', 'source_y', 'check_status', 'check_src'], inplace = True)
#rename cols
col_names = list(data.columns)
col_names[1] = 'status'
col_names[2] = 'source'
data.columns = col_names
data.head()

In [ ]:
#parse date_time
data['date_time'] = pd.to_datetime(data['date_time'])
data.info()

In [ ]:
#decompose datetime into components
data['year'] = data['date_time'].dt.year
data['month'] = data['date_time'].dt.month
data['day'] = data['date_time'].dt.day
data['hour'] = data['date_time'].dt.hour
data['date'] = data['date_time'].dt.date
data['day_week'] = data['date_time'].dt.day_of_week
#convert bools to 1-0 binary
for col in ['status', 'cvv_provided', 'chargeback']:
    data[f'{col}_num'] = data[col].astype(int)
#Add is_declined variable
data['is_declined'] = data['state'].map({'DECLINED': 1, 'ACCEPTED': 0})
#function to convert currencies to USD
def get_usd(amount_local, currency, rates_dict):
    rates_dict = json.loads(rates_dict)
    amount_usd = amount_local / (rates_dict[currency])
    return amount_usd
data['amount_usd'] = [
    get_usd(amount, currency, rates) for amount, currency, rates in zip(
        list(data['amount']), list(data['currency']), list(data['rates'])
    )
]
data.info()

In [ ]:
#distributions  of USD amount 
fig = px.box(data, y = 'amount_usd', color = 'is_declined')
fig.show()

In [ ]:
#Group by country and currency
def group_for_task(list_col_to_group, dataframe):
    groupped = dataframe.groupby(list_col_to_group).agg({
        "is_declined": ['count', 'sum']
    }).reset_index()
    col_names = list_col_to_group + ['count_all', 'count_declined']
    groupped.columns = col_names
    groupped['shared_declined'] = round(groupped['count_declined'] / groupped['count_all'], 3)
    return groupped
    
country_curr = group_for_task(['country', 'currency'], data)
country_curr.head()

Country + currency does not give and insights :( 

In [ ]:
#timeseries by date by country
date_country = group_for_task(['hour', 'country'], data)
fig = px.line(date_country, x = 'hour', y = 'shared_declined', color = 'country')
fig.show()

In [ ]:
cvv_country = group_for_task(['cvv_provided_num', 'country'], data)
cvv_country

In [ ]:
data.head()

In [ ]:
#timeseries by date
date_country = group_for_task(['date'], data)
fig = px.line(date_country, x = 'date', y = 'shared_declined')
fig.show()

In [ ]:
day_week_agg =  group_for_task(['date'], data)
day_week_agg.sort_values(by = 'shared_declined', ascending = False, inplace = True)
# fig = px.box(day_week_agg, y = 'count_all', color = 'country')
# fig.show()
day_week_agg

In [ ]:
data.info()